In [5]:
import pandas as pd
import numpy as np
from functools import reduce
from unicodedata import normalize

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import train_test_split, KFold,cross_val_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import SGDRegressor

In [2]:

def filter_for_test_presence(dataframe):
    dataframe = dataframe[(dataframe['TP_PRESENCA_CN'] == 1) & (dataframe['TP_PRESENCA_CH'] == 1) \
         & (dataframe['TP_PRESENCA_LC'] == 1) & (dataframe['TP_PRESENCA_MT'] == 1)]
    dataframe = dataframe.drop(['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT'] ,axis=1)
    return dataframe



def create_target_column(dataframe, column_name, cols_to_use, method='mean'):
    if method == 'mean':
        dataframe[column_name] = dataframe[cols_to_use].mean(axis=1)
    elif method == 'sum':
        dataframe[column_name] = dataframe[cols_to_use].sum(axis=1)
    dataframe = dataframe.drop(cols_to_use, axis=1)
    return dataframe


def create_enem_dataframe(filepath, cols_to_use, chunksize=None, sample_size=None):
    
    dataframe = []
    
    if sample_size:
        sample = pd.read_csv(filepath, sep=';', encoding='latin-1',  usecols=cols_to_use, nrows=sample_size)
        sample = sample.dropna(subset=['CO_MUNICIPIO_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC'])
        sample = filter_for_test_presence(sample)
        dataframe.append(sample)
        
    else:
        chunks = pd.read_csv(filepath, sep=';', encoding='latin-1',  usecols=cols_to_use, chunksize=chunksize)
        for chunk in chunks:
            chunk = chunk.dropna(subset=['CO_MUNICIPIO_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC'])
            chunk = filter_for_test_presence(chunk)
            dataframe.append(chunk)
            
    dataframe = pd.concat(dataframe)
    dataframe = create_target_column(dataframe, 'target', 
                                     ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT'], 'mean')
    
    return dataframe
            
    

    
cols_enem_to_use = ['NU_INSCRICAO','TP_FAIXA_ETARIA','TP_SEXO','TP_ESTADO_CIVIL','TP_COR_RACA','TP_ST_CONCLUSAO',
                    'TP_ESCOLA','TP_ENSINO','IN_TREINEIRO','CO_MUNICIPIO_ESC','CO_UF_ESC',
                    'TP_DEPENDENCIA_ADM_ESC','TP_LOCALIZACAO_ESC','TP_SIT_FUNC_ESC','CO_MUNICIPIO_PROVA', 'TP_PRESENCA_CN','TP_PRESENCA_CH',
                    'TP_PRESENCA_LC','TP_PRESENCA_MT','CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT',
                    'NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'TP_ESTADO_CIVIL']

enem_sample_size=1_000_000
        
enem_raw = create_enem_dataframe(filepath="../dados/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", 
                                 cols_to_use=cols_enem_to_use, chunksize=500_000)
saeb_munic_raw = pd.read_excel("../dados/saeb/Resultados_Saeb_2019_Brasil_Estados_Municipios.xlsx", sheet_name='Municípios')
      

## Saeb

In [6]:
def create_saeb_dataframe(filepath, label_dict_adm=None, label_dict_region=None):
    
    if label_dict_adm is None:
        label_dict_adm = {"federal":1, "estadual":2, "municipal":3, "privada":4}
        
    if label_dict_region is None:
        label_dict_region = {"urbana":1, "rural":2}
    
    saeb_df = pd.read_excel(filepath, sheet_name='Municípios')
    saeb_df['DEPENDENCIA_ADM'] = saeb_df['DEPENDENCIA_ADM'].apply(lambda x: 
                                                                  label_dict_adm[x.lower()] if 'total' not in x.lower() else None)
    saeb_df['LOCALIZACAO'] = saeb_df['LOCALIZACAO'].apply(lambda x: 
                                                          label_dict_region[x.lower()] if 'total' not in x.lower() else None)
    saeb_df = saeb_df.dropna(subset=['DEPENDENCIA_ADM','LOCALIZACAO'])
    return saeb_df


def fill_na_values_mean(data, cat_column, cols_to_use):
    
    data_to_return = data.copy()
    for c in cols_to_use:
        grouped = data_to_return.groupby(cat_column)[c].transform("mean")
        data_to_return[c] = data_to_return[c].fillna(grouped)
        data_to_return[c] = data_to_return[c].fillna(data_to_return[c].mean())
    
    return data_to_return


def transform_to_string(x):
    return str(int(x)) if not np.isnan(x) else ''

def create_merge_key(dataframe, column_key_name=None, **kwargs):
    munic_key = kwargs['munic_key']
    adm_key = kwargs['adm_key']
    local_key = kwargs['local_key']
    
    if column_key_name is None:
        column_key_name = 'key_escola'
    
    dataframe[column_key_name] = dataframe.apply(lambda row: transform_to_string(row[munic_key]) + \
                                            transform_to_string(row[adm_key]) + \
                                            transform_to_string(row[local_key]), axis=1)
    return dataframe

In [8]:
enem = enem_raw.copy()
dict_cod_adm = {"federal":1, "estadual":2, "municipal":3, "privada":4}
dict_cod_regiao = {"urbana":1, "rural":2}
saeb_munic = create_saeb_dataframe("../dados/saeb/Resultados_Saeb_2019_Brasil_Estados_Municipios.xlsx",
                                  label_dict_adm=dict_cod_adm, label_dict_region=dict_cod_regiao)
saeb_munic.head()

,CO_UF,NO_UF,CO_MUNICIPIO,NO_MUNICIPIO,DEPENDENCIA_ADM,LOCALIZACAO,MEDIA_5_LP,MEDIA_5_MT,MEDIA_9_LP,MEDIA_9_MT,...,nivel_1_MT12,nivel_2_MT12,nivel_3_MT12,nivel_4_MT12,nivel_5_MT12,nivel_6_MT12,nivel_7_MT12,nivel_8_MT12,nivel_9_MT12,nivel_10_MT12
1,11,Rondônia,1100015,Alta Floresta D'Oeste,2.0,1.0,210.48,233.47,268.37,273.59,...,13.63,12.98,22.16,16.69,14.41,5.81,1.90,0.00,0.0,0.0
2,11,Rondônia,1100015,Alta Floresta D'Oeste,3.0,2.0,189.48,203.69,254.80,252.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,Rondônia,1100015,Alta Floresta D'Oeste,3.0,1.0,180.19,196.64,252.15,255.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,11,Rondônia,1100023,Ariquemes,2.0,2.0,NaN,NaN,235.50,258.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,11,Rondônia,1100023,Ariquemes,2.0,1.0,211.65,223.35,266.19,267.39,...,14.30,21.89,18.97,18.88,8.80,4.81,0.69,0.14,0.0,0.0


In [9]:

enem = create_merge_key(enem, munic_key='CO_MUNICIPIO_ESC', adm_key='TP_DEPENDENCIA_ADM_ESC', local_key='TP_LOCALIZACAO_ESC')

cols_to_use = [c for c in saeb_munic.columns[6:] if 'MEDIA' in c]
saeb_munic = fill_na_values_mean(saeb_munic, 'CO_MUNICIPIO', cols_to_use)
saeb_munic = create_merge_key(saeb_munic,munic_key='CO_MUNICIPIO', adm_key='DEPENDENCIA_ADM', local_key='LOCALIZACAO')
saeb_munic_means = saeb_munic[['MEDIA_5_LP', 'MEDIA_5_MT', 'MEDIA_9_LP', 'MEDIA_9_MT', 'MEDIA_12_LP', 'MEDIA_12_MT', 'key_escola']].copy()


In [10]:
enem_saeb = enem.merge(saeb_munic_means, on='key_escola', how='left')
enem_saeb = enem_saeb.dropna(subset=['MEDIA_12_LP', 'MEDIA_12_MT'])

## Censo Escolar

In [11]:
dict_censo_indicators = {"taxa_rendimento": ("../dados/censo_escolar_2020/indicadores_educacionais/tx_rend_municipios_2020/tx_rend_municipios_2020_simplified.csv",
                                            "CENSO_RENDIMENTO_"),
                        "esforco_docente":("../dados/censo_escolar_2020/indicadores_educacionais/IED_2020_MUNICIPIOS/esforco_docente_municipios_2020_simplified.csv", 
                                           "CENSO_ESFORCO_DOCENTE_"),
                        "horas_dia":("../dados/censo_escolar_2020/indicadores_educacionais/HAD_2020_MUNICIPIOS/horas_aula_municipios_2020_simplified.csv", 
                                            "CENSO_HORAS_AULA_"),
                        "alunos_aula":("../dados/censo_escolar_2020/indicadores_educacionais/ATU_2020_MUNICIPIOS/alunos_turma_municipios_2020_simplified.csv", 
                                       "CENSO_ALUNOS_TURMA"),
                        "distorcao_idade_serie":("../dados/censo_escolar_2020/indicadores_educacionais/TDI_2020_MUNICIPIOS/taxa_distorcao_idade_serie_municipios_2020_simplified.csv",
                                                "CENSO_DIST_IDADE_SERIE")}



def normalize_string(x):
    x_normed = normalize('NFKD', x).encode('ASCII','ignore').decode('ASCII')
    x_normed = x_normed.replace(" ", "_")
    return x_normed.upper()



def process_census_indicator_info(filepath, prefix,remove_columns,  key_name='key_escola'):
    indicator_info = pd.read_csv(filepath)
    indicator_info.columns =  [prefix + normalize_string(c) for c in indicator_info.columns]
    
    adm_column_name = "DEPENDENCIA_ADMINISTRATIVA"
    location_column_name = "LOCALIZACAO"
    city_code_column_name = "CODIGO_DO_MUNICIPIO"
    uf_column_name = "UF"
    
    indicator_info[f'{prefix}{adm_column_name}'] = indicator_info[f'{prefix}{adm_column_name}'].apply(lambda x: dict_cod_adm.get(x.lower()))
    indicator_info[f'{prefix}{location_column_name}'] = indicator_info[f'{prefix}{location_column_name}'].apply(lambda x: dict_cod_regiao.get(x.lower()))

    indicator_info = indicator_info.dropna(subset=[f'{prefix}{adm_column_name}', f'{prefix}{location_column_name}'])

    indicator_info[key_name] = indicator_info.apply(lambda row: transform_to_string(row[f'{prefix}{city_code_column_name}']) + \
                                                transform_to_string(row[f'{prefix}{adm_column_name}']) + \
                                                transform_to_string(row[f'{prefix}{location_column_name}']), axis=1)
    
    if remove_columns:
        for i in [adm_column_name, location_column_name, city_code_column_name, uf_column_name, 'MUNICIPIO',  'REGIAO', 'SIGLA']:
            for col in indicator_info.columns:
                if i in col:
                    indicator_info = indicator_info.drop(col, axis=1)
    
    return indicator_info


school_census_df = []

for p, ind in enumerate(dict_censo_indicators):
    filepath, prefix = dict_censo_indicators[ind]
    remove_dup_cols = False if p == 0 else True
    census_indicator_info = process_census_indicator_info(filepath, prefix, remove_dup_cols)
    census_indicator_info = census_indicator_info.replace("--", np.nan)
    school_census_df.append(census_indicator_info)
    
    
school_census_df = reduce(lambda df1,df2: df1.merge(df2, on='key_escola', how='left'), school_census_df)
school_census_df.head()

,CENSO_RENDIMENTO_REGIAO,CENSO_RENDIMENTO_UF,CENSO_RENDIMENTO_CODIGO_DO_MUNICIPIO,CENSO_RENDIMENTO_NOME_DO_MUNICIPIO,CENSO_RENDIMENTO_LOCALIZACAO,CENSO_RENDIMENTO_DEPENDENCIA_ADMINISTRATIVA,CENSO_RENDIMENTO_APROVACAO-EF-TOTAL,CENSO_RENDIMENTO_APROVACAO-EF-ANOS_INICIAIS,CENSO_RENDIMENTO_APROVACAO-EF-ANOS_FINAIS,CENSO_RENDIMENTO_APROVACAO-EM-TOTAL__,...,CENSO_HORAS_AULA_EF-ANOS_FINAIS,CENSO_HORAS_AULA_EM-TOTAL,CENSO_ALUNOS_TURMAEF-TOTAL,CENSO_ALUNOS_TURMAEF-ANOS_INICIAIS,CENSO_ALUNOS_TURMAEF-ANOS_FINAIS,CENSO_ALUNOS_TURMAEM-TOTAL,CENSO_DIST_IDADE_SERIEEF-TOTAL,CENSO_DIST_IDADE_SERIEEF-ANOS_INICIAIS,CENSO_DIST_IDADE_SERIEEF-ANOS_FINAIS,CENSO_DIST_IDADE_SERIEEM-TOTAL
0,Norte,RO,1100015,Alta Floresta D'Oeste,1.0,2.0,97.1,93.7,97.8,98.4,...,4.0,5.2,26.6,23.0,27.6,16.6,18.6,10.9,20.4,21.2
1,Norte,RO,1100015,Alta Floresta D'Oeste,2.0,2.0,0.0,0.0,0.0,86.2,...,4.0,4.0,5.9,3.7,11.0,7.3,41.5,20.2,58.3,84.5
2,Norte,RO,1100015,Alta Floresta D'Oeste,1.0,3.0,98.7,99.1,97.4,NaN,...,4.0,NaN,20.1,21.5,17.6,NaN,8.6,3.6,25.2,NaN
3,Norte,RO,1100015,Alta Floresta D'Oeste,2.0,3.0,97.1,98.1,95.9,NaN,...,4.0,NaN,16.0,18.5,16.3,NaN,13.5,5.5,22.4,NaN
4,Norte,RO,1100023,Ariquemes,2.0,1.0,NaN,NaN,NaN,68.1,...,NaN,9.3,NaN,NaN,NaN,28.6,NaN,NaN,NaN,12.6


In [12]:
columns_to_use = [c for c in school_census_df.columns[6:] if 'CENSO' in c]
for c in columns_to_use:
    school_census_df[c] = school_census_df[c].astype(float)
school_census_df = fill_na_values_mean(school_census_df, "CENSO_RENDIMENTO_CODIGO_DO_MUNICIPIO", columns_to_use)

school_census_df.head()

,CENSO_RENDIMENTO_REGIAO,CENSO_RENDIMENTO_UF,CENSO_RENDIMENTO_CODIGO_DO_MUNICIPIO,CENSO_RENDIMENTO_NOME_DO_MUNICIPIO,CENSO_RENDIMENTO_LOCALIZACAO,CENSO_RENDIMENTO_DEPENDENCIA_ADMINISTRATIVA,CENSO_RENDIMENTO_APROVACAO-EF-TOTAL,CENSO_RENDIMENTO_APROVACAO-EF-ANOS_INICIAIS,CENSO_RENDIMENTO_APROVACAO-EF-ANOS_FINAIS,CENSO_RENDIMENTO_APROVACAO-EM-TOTAL__,...,CENSO_HORAS_AULA_EF-ANOS_FINAIS,CENSO_HORAS_AULA_EM-TOTAL,CENSO_ALUNOS_TURMAEF-TOTAL,CENSO_ALUNOS_TURMAEF-ANOS_INICIAIS,CENSO_ALUNOS_TURMAEF-ANOS_FINAIS,CENSO_ALUNOS_TURMAEM-TOTAL,CENSO_DIST_IDADE_SERIEEF-TOTAL,CENSO_DIST_IDADE_SERIEEF-ANOS_INICIAIS,CENSO_DIST_IDADE_SERIEEF-ANOS_FINAIS,CENSO_DIST_IDADE_SERIEEM-TOTAL
0,Norte,RO,1100015,Alta Floresta D'Oeste,1.0,2.0,97.10,93.700,97.80,98.4,...,4.00,5.2,26.60,23.00,27.6,16.60,18.60,10.900,20.40,21.20
1,Norte,RO,1100015,Alta Floresta D'Oeste,2.0,2.0,0.00,0.000,0.00,86.2,...,4.00,4.0,5.90,3.70,11.0,7.30,41.50,20.200,58.30,84.50
2,Norte,RO,1100015,Alta Floresta D'Oeste,1.0,3.0,98.70,99.100,97.40,92.3,...,4.00,4.6,20.10,21.50,17.6,11.95,8.60,3.600,25.20,52.85
3,Norte,RO,1100015,Alta Floresta D'Oeste,2.0,3.0,97.10,98.100,95.90,92.3,...,4.00,4.6,16.00,18.50,16.3,11.95,13.50,5.500,22.40,52.85
4,Norte,RO,1100023,Ariquemes,2.0,1.0,96.84,99.075,96.66,68.1,...,4.44,9.3,24.14,21.95,25.2,28.60,16.12,8.775,22.18,12.60


In [13]:
enem_final = enem_saeb.merge(school_census_df, how='left', on='key_escola')
enem_final.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,...,CENSO_HORAS_AULA_EF-ANOS_FINAIS,CENSO_HORAS_AULA_EM-TOTAL,CENSO_ALUNOS_TURMAEF-TOTAL,CENSO_ALUNOS_TURMAEF-ANOS_INICIAIS,CENSO_ALUNOS_TURMAEF-ANOS_FINAIS,CENSO_ALUNOS_TURMAEM-TOTAL,CENSO_DIST_IDADE_SERIEEF-TOTAL,CENSO_DIST_IDADE_SERIEEF-ANOS_INICIAIS,CENSO_DIST_IDADE_SERIEEF-ANOS_FINAIS,CENSO_DIST_IDADE_SERIEEM-TOTAL
0,200001908998,2,M,1,3,2,2,1.0,0,3547304.0,...,5.3,4.4,27.5,26.4,29.1,30.3,7.2,3.3,12.2,13.3
1,200003209996,3,F,1,3,2,2,1.0,0,1302009.0,...,4.3,8.6,26.7,29.1,24.8,38.8,10.5,11.1,9.8,22.2
2,200005725669,2,F,1,3,2,2,1.0,0,5108402.0,...,4.1,4.1,26.3,24.5,27.0,30.2,11.7,6.7,13.5,25.8
3,200004462096,2,F,1,1,2,2,1.0,0,3513603.0,...,5.5,5.3,28.4,16.7,28.4,33.0,15.7,3.2,15.7,16.2
4,200005300966,4,M,1,2,2,2,1.0,0,2202000.0,...,4.7,6.3,31.8,18.7,31.8,29.5,22.0,15.4,22.0,44.7


## Variables Preprocessing

In [14]:

variables_for_onehot_encoder = ['TP_SEXO', 'TP_COR_RACA', 'TP_ESCOLA', 'TP_ENSINO',
                                'IN_TREINEIRO', 'CO_PROVA_CN',  'CO_PROVA_CH', 'CO_PROVA_LC','CO_PROVA_MT']
variables_for_target_encoder = ['CO_MUNICIPIO_ESC']



features = enem_final.copy()

most_present_municipalities = features['CO_MUNICIPIO_ESC'].value_counts().iloc[:1500]
most_present_municipalities_codes = most_present_municipalities.index

features = features[features['CO_MUNICIPIO_ESC'].apply(lambda x: x in most_present_municipalities_codes)]
features = features.reset_index(drop=True)

values = features[variables_for_onehot_encoder]
encoder = OneHotEncoder()
result = encoder.fit_transform(values)

onehot_column_names = []

for p,i in enumerate(variables_for_onehot_encoder):
    current_cat_values = encoder.categories_[p]
    current_cat_values = [i+'_'+str(j) for j in current_cat_values]
    onehot_column_names += current_cat_values
    
df_encoded = pd.DataFrame(data=result.toarray(), columns=onehot_column_names)
features = features.drop(variables_for_onehot_encoder, axis=1)
features =  features.join(df_encoded)


##target
target_encoder = TargetEncoder()
encoded_var, target = features['CO_MUNICIPIO_ESC'].astype('str'), features['target']
encoded_var = target_encoder.fit_transform(encoded_var, target).iloc[:,0].values

features['CO_MUNICIPIO_ESC'] = encoded_var


features['TP_ST_CONCLUSAO'] = features['TP_ST_CONCLUSAO'].apply(lambda x: 1 if x==1 else 0)
features['TP_ESTADO_CIVIL'] = features['TP_ESTADO_CIVIL'].apply(lambda x: 1 if x==1 else 0)


cols_to_drop = ['NU_INSCRICAO','CO_UF_ESC','TP_DEPENDENCIA_ADM_ESC','TP_LOCALIZACAO_ESC','TP_SIT_FUNC_ESC','CO_MUNICIPIO_PROVA', 'key_escola',
               'CENSO_RENDIMENTO_REGIAO','CENSO_RENDIMENTO_UF',  'CENSO_RENDIMENTO_CODIGO_DO_MUNICIPIO',
               'CENSO_RENDIMENTO_NOME_DO_MUNICIPIO', 'CENSO_RENDIMENTO_LOCALIZACAO',
               'CENSO_RENDIMENTO_DEPENDENCIA_ADMINISTRATIVA']
features = features.drop(cols_to_drop, axis=1)
print(f'Final DataFrame Shape: {features.shape}')

/home/thiago/MAC0460-projeto/venv/lib/python3.9/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/thiago/MAC0460-projeto/venv/lib/python3.9/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Final DataFrame Shape: (296546, 111)


## Validation Set

In [15]:
X,y = features.drop(['target'], axis=1), features['target']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=40)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=40)



### Scaling Variables (for algorithms using GD)

In [16]:
def find_continuous_columns(dataframe):
    continuous_cols = []
    for c in dataframe.columns:
        unique_vals = dataframe[c].nunique()
        if unique_vals > 20:
            continuous_cols.append(c)
    return continuous_cols


def normalize_continuous_columns(dataframe):
    continuous_columns = find_continuous_columns(dataframe)
    scaler = StandardScaler()
    dataframe[continuous_columns] = scaler.fit_transform(dataframe[continuous_columns])
    return dataframe, scaler

In [17]:
X_train, scaler = normalize_continuous_columns(X_train)

## Model Training

### Linear Regression

In [ ]:
gd_reg = SGDRegressor()
cross_val_score(estimator=gd_reg, X=X_train, y=y_train, scoring='neg_mean_absolute_error', cv=3, verbose=2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   1.8s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] END .................................................... total time=   2.3s


### Random Forest

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=200)

cross_val_score(estimator=rf_regressor, X=X_train, y=y_train, scoring='neg_mean_absolute_error', cv=3, verbose=2)